In [115]:
from tslearn.clustering import TimeSeriesKMeans
import pickle
import matplotlib.pyplot as plt
import numpy as np
import scipy.spatial.distance as dist

from sklearn.metrics import davies_bouldin_score
from sklearn.utils import check_random_state
from sklearn.utils import check_X_y
from sklearn.utils import _safe_indexing
from sklearn.metrics.pairwise import pairwise_distances_chunked
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.preprocessing import LabelEncoder


In [116]:
def check_number_of_labels(n_labels, n_samples):
    """Check that number of labels are valid.

    Parameters
    ----------
    n_labels : int
        Number of labels.

    n_samples : int
        Number of samples.
    """
    if not 1 < n_labels < n_samples:
        raise ValueError(
            "Number of labels is %d. Valid values are 2 to n_samples - 1 (inclusive)"
            % n_labels
        )

def davies_bouldin_score(X, labels,centroids):
    """Compute the Davies-Bouldin score.

    The score is defined as the average similarity measure of each cluster with
    its most similar cluster, where similarity is the ratio of within-cluster
    distances to between-cluster distances. Thus, clusters which are farther
    apart and less dispersed will result in a better score.

    The minimum score is zero, with lower values indicating better clustering.

    Read more in the :ref:`User Guide <davies-bouldin_index>`.

    .. versionadded:: 0.20

    Parameters
    ----------
    X : array-like of shape (n_samples, n_features)
        A list of ``n_features``-dimensional data points. Each row corresponds
        to a single data point.

    labels : array-like of shape (n_samples,)
        Predicted labels for each sample.

    Returns
    -------
    score: float
        The resulting Davies-Bouldin score.

    References
    ----------
    .. [1] Davies, David L.; Bouldin, Donald W. (1979).
       `"A Cluster Separation Measure"
       <https://ieeexplore.ieee.org/document/4766909>`__.
       IEEE Transactions on Pattern Analysis and Machine Intelligence.
       PAMI-1 (2): 224-227
    """
    X, labels = check_X_y(X, labels)
    le = LabelEncoder()
    labels = le.fit_transform(labels)
    n_samples, _ = X.shape
    n_labels = len(le.classes_)
    check_number_of_labels(n_labels, n_samples)

    intra_dists = np.zeros(n_labels)
    #centroids = np.zeros((n_labels, len(X[0])), dtype=float)
    for k in range(n_labels):
        cluster_k = _safe_indexing(X, labels == k)
        #centroid = cluster_k.mean(axis=0)
        #centroids[k] = centroid
        intra_dists[k] = np.average(pairwise_distances(cluster_k, [centroids[k]]))

    centroid_distances = pairwise_distances(centroids)

    if np.allclose(intra_dists, 0) or np.allclose(centroid_distances, 0):
        return 0.0

    centroid_distances[centroid_distances == 0] = np.inf
    combined_intra_dists = intra_dists[:, None] + intra_dists
    scores = np.max(combined_intra_dists / centroid_distances, axis=1)
    return np.mean(scores)


In [117]:
with open('./pickles/allPixelNDVIPoly.pickle', 'rb') as handle:
    allPixelNDVIPoly3 = pickle.load(handle)

with open('./pickles/newResa3.pickle', 'rb') as handle:
    newResa3 = pickle.load(handle)

with open('./pickles/allPixelNDVIPoly4.pickle', 'rb') as handle:
    allPixelNDVIPoly4 = pickle.load(handle)

with open('./pickles/newResa4.pickle', 'rb') as handle:
    newResa4 = pickle.load(handle)

with open('./pickles/allPixelNDVIPoly6.pickle', 'rb') as handle:
    allPixelNDVIPoly6 = pickle.load(handle)

with open('./pickles/newResa6.pickle', 'rb') as handle:
    newResa6 = pickle.load(handle)

num_cluster = 2

In [118]:
allPixelNDVIPoly3 = allPixelNDVIPoly3[(newResa3<=11000) & (newResa3 >= 4000),:]
newResa3 = newResa3[(newResa3<=11000) & (newResa3 >= 4000)]

allPixelNDVIPoly4 = allPixelNDVIPoly4[(newResa4<=11000) & (newResa4 >= 4000),:]
newResa4 = newResa4[(newResa4<=11000) & (newResa4 >= 4000)]
print(len(newResa4))

allPixelNDVIPoly6 = allPixelNDVIPoly6[(newResa6<=11000) & (newResa6 >= 4000),:]
newResa6 = newResa6[(newResa6<=11000) & (newResa6 >= 4000)]
print(len(newResa6))

1065
1700


In [119]:
# k-means su tutto l'asse temporale 

km_one3 = TimeSeriesKMeans(n_clusters=num_cluster, metric="euclidean", max_iter=100,random_state=0)
y_pred_one3 = km_one3.fit_predict(allPixelNDVIPoly3)

km_one4 = TimeSeriesKMeans(n_clusters=num_cluster, metric="euclidean", max_iter=100,random_state=0)
y_pred_one4 = km_one4.fit_predict(allPixelNDVIPoly4)

km_one6 = TimeSeriesKMeans(n_clusters=num_cluster, metric="euclidean", max_iter=100,random_state=0)
y_pred_one6 = km_one6.fit_predict(allPixelNDVIPoly6)

In [120]:
print(len(allPixelNDVIPoly3[y_pred_one3 == 0][0]))
print(len(km_one3.cluster_centers_[0].ravel()))
print(np.linalg.norm(allPixelNDVIPoly3[y_pred_one3 == 0][0] - km_one3.cluster_centers_[0].ravel()))
print(dist.euclidean(allPixelNDVIPoly3[y_pred_one3 == 0][0],km_one3.cluster_centers_[0].ravel()))

150
150
1.4875373563042251
1.4875373563042251


In [121]:
distancefromC = []
centroids = []
for cluster in range(0,num_cluster):
    distancefromC.append([])
    #
    ClusterTs = allPixelNDVIPoly3[y_pred_one3 == cluster]
    centroid = np.array(km_one3.cluster_centers_[cluster].ravel())
    centroids = np.tile(centroid,(ClusterTs.shape[0]))
    print(centroid.shape)
    print(centroids.shape,ClusterTs.shape)
    #for timeSeries in allPixelNDVIPoly3[y_pred_one3 == cluster]:
    #    distancefromC[-1].append(dist.euclidean(timeSeries,km_one3.cluster_centers_[cluster].ravel()))

(150,)
(79950,) (533, 150)
(150,)
(140700,) (938, 150)


# Coesione del TS

In [122]:
for cluster in range(0,num_cluster):
    print("cluster"+str(cluster))
    print(np.array(distancefromC[cluster]).mean())
    print(np.array(distancefromC[cluster]).std())



cluster0
nan
nan
cluster1
nan
nan


C:\Users\Leo\AppData\Local\Temp\ipykernel_200\468600622.py:3: RuntimeWarning: Mean of empty slice.
  print(np.array(distancefromC[cluster]).mean())
c:\Users\Leo\.conda\envs\bigdata\lib\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
c:\Users\Leo\.conda\envs\bigdata\lib\site-packages\numpy\core\_methods.py:265: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
c:\Users\Leo\.conda\envs\bigdata\lib\site-packages\numpy\core\_methods.py:223: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
c:\Users\Leo\.conda\envs\bigdata\lib\site-packages\numpy\core\_methods.py:257: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


# Davies–Bouldin index

In [123]:
#Rese 3 
centroids = []
for cluster in range(0,num_cluster):
    centroids.append(km_one3.cluster_centers_[cluster].ravel())
centroids = np.array(centroids)
print("indice dv del campo",3)
print(davies_bouldin_score(allPixelNDVIPoly3,y_pred_one3,centroids))

#Resa 4
centroids = []
for cluster in range(0,num_cluster):
    centroids.append(km_one4.cluster_centers_[cluster].ravel())
centroids = np.array(centroids)
print("indice dv del campo",4)
print(davies_bouldin_score(allPixelNDVIPoly4,y_pred_one4,centroids))

#Resa 6
centroids = []
for cluster in range(0,num_cluster):
    centroids.append(km_one6.cluster_centers_[cluster].ravel())
centroids = np.array(centroids)
print("indice dv del campo",6)
print(davies_bouldin_score(allPixelNDVIPoly6,y_pred_one6,centroids))

indice dv del campo 3
0.9182155745206122
indice dv del campo 4
0.6959551693972861
indice dv del campo 6
0.35611207971866476
